# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

# Setup

In [3]:
!kubectl create namespace seldon

namespace/seldon created


In [4]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [5]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


# Install Helm

In [6]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [7]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


## Start seldon-core

In [12]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Sun Oct  7 10:02:44 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     0s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/Pod(related)
NAME                                         READY  STATUS       RESTARTS  AGE
seldon-spartakus-volunteer-7d9895b6fb-gz6kh  0/1    Terminating  0         2m


NOTES:
NOTES: TODO




In [13]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon  --set ambassador.enabled=true

NAME:   seldon-core
LAST DEPLOYED: Sun Oct  7 10:02:51 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/ClusterRole
NAME        AGE
seldon-crd  1s

==> v1/ClusterRoleBinding
NAME    AGE
seldon  1s

==> v1beta1/Role
NAME          AGE
seldon-local  1s
ambassador    1s

==> v1beta1/RoleBinding
NAME        AGE
ambassador  1s

==> v1/Service
NAME                          TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)                        AGE
seldon-core-ambassador-admin  NodePort   10.105.58.221  <none>       8877:30280/TCP                 1s
seldon-core-ambassador        NodePort   10.106.202.84  <none>       8080:31373/TCP                 1s
seldon-core-seldon-apiserver  NodePort   10.110.76.142  <none>       8080:30284/TCP,5000:30036/TCP  1s
seldon-core-redis             ClusterIP  10.103.18.74   <none>       6379/TCP                       1s

==> v1/Pod(related)
NAME                                                 READY  STATUS             RESTARTS  AGE
seldon-core-amba

In [14]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador

deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

Install gRPC modules for the prediction protos.

In [15]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

## Serve Single Model

In [29]:
!helm install ../helm-charts/seldon-single-model --name mymodel

NAME:   mymodel
LAST DEPLOYED: Sun Oct  7 10:16:01 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME     AGE
mymodel  0s




In [30]:
!helm template ../helm-charts/seldon-single-model --name mymodel | pygmentize -l json

---
# Source: seldon-single-model/templates/model.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mymodel"
    },
    "spec": {
        "name": "mymodel",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                        "terminationGracePeriodSecond

In [32]:
!kubectl rollout status deploy/mymodel-mymodel-classifier-0
!kubectl rollout status deploy/mymodel-mymodel-svc-orch

deployment "mymodel-mymodel-classifier-0" successfully rolled out
deployment "mymodel-mymodel-svc-orch" successfully rolled out


### Get predictions

In [33]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [34]:
rest_request_ambassador("mymodel",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "hj5j8rj4nnf78o70tji6jmcsjc",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.06517529431696042]
    }
  }
}


#### gRPC Request

In [35]:
grpc_request_ambassador("mymodel",API_AMBASSADOR)

meta {
  puid: "gjb88eohld41rfhblbo0cc3l68"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07908782658235097
  }
}



In [36]:
!helm delete mymodel --purge

release "mymodel" deleted


## Serve AB Test

In [37]:
!helm install ../helm-charts/seldon-abtest --name myabtest

NAME:   myabtest
LAST DEPLOYED: Sun Oct  7 10:22:18 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME      AGE
myabtest  0s




In [38]:
!helm template ../helm-charts/seldon-abtest --name myabtest | pygmentize -l json

---
# Source: seldon-abtest/templates/ab_test_1pod.json


---
# Source: seldon-abtest/templates/ab_test_2pods.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
	"labels": {
	    "app": "seldon"
	},
	"name": "myabtest"
    },
    "spec": {
	"name": "myabtest",
	"predictors": [
	    {
		"name": "abtest",
		"replicas": 1,
		"componentSpecs": [{
		    "spec": {
			"containers": [
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-1",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }],
			"terminationGracePeriodSeconds": 20
		    }},
	        {
		    "metadata":{
			"labels":{
			    "version":"v2"
			}
		    },    
			"spec":{
			    "containers":[
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-2",
				"resources": {
				    "requests": {
					"memory": "1M

In [40]:
!kubectl rollout status deploy/myabtest-abtest-classifier-1-0
!kubectl rollout status deploy/myabtest-abtest-classifier-1-0
!kubectl rollout status deploy/myabtest-abtest-svc-orch 

deployment "myabtest-abtest-classifier-1-0" successfully rolled out
deployment "myabtest-abtest-classifier-1-0" successfully rolled out
deployment "myabtest-abtest-svc-orch" successfully rolled out


### Get predictions

In [41]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [43]:
rest_request_ambassador("myabtest",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "79quptt277oh69vr9at995jldm",
    "tags": {
    },
    "routing": {
      "myabtest": 1
    },
    "requestPath": {
      "myabtest": "",
      "classifier-2": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.10107939865261878]
    }
  }
}


#### gRPC Request

In [44]:
grpc_request_ambassador("myabtest",API_AMBASSADOR)

meta {
  puid: "61sb5ijc5s2etcc05jrnac3tg3"
  routing {
    key: "myabtest"
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "myabtest"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08168058489600766
  }
}



In [45]:
!helm delete myabtest --purge

release "myabtest" deleted


## Serve Multi-Armed Bandit

In [48]:
!helm install ../helm-charts/seldon-mab --name mymab

NAME:   mymab
LAST DEPLOYED: Sun Oct  7 10:28:29 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME   AGE
mymab  0s




In [49]:
!helm template ../helm-charts/seldon-mab --name mymab | pygmentize -l json

---
# Source: seldon-mab/templates/mab.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
	"labels": {
	    "app": "seldon"
	},
	"name": "mymab"
    },
    "spec": {
	"name": "mymab",
	"predictors": [
	    {
		"name": "abtest",
		"replicas": 1,
		"componentSpecs": [{
		    "spec": {
			"containers": [
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-1",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }],
			"terminationGracePeriodSeconds": 20
		    }},
	        {
		    "metadata":{
			"labels":{
			    "version":"v2"
			}
		    },    
			"spec":{
			    "containers":[
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-2",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }
			],
			"terminationGracePeriodSeconds": 20
			}


In [51]:
!kubectl rollout status deploy/mymab-abtest-classifier-1-0
!kubectl rollout status deploy/mymab-abtest-classifier-2-1
!kubectl rollout status deploy/mymab-abtest-eg-router-2 

deployment "mymab-abtest-classifier-1-0" successfully rolled out
deployment "mymab-abtest-classifier-2-1" successfully rolled out
deployment "mymab-abtest-eg-router-2" successfully rolled out


### Get predictions

In [52]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [54]:
rest_request_ambassador("mymab",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "jirg0r3va3frg6dvggc0bt3bbh",
    "tags": {
    },
    "routing": {
      "eg-router": 0
    },
    "requestPath": {
      "eg-router": "seldonio/mab_epsilon_greedy:1.1",
      "classifier-1": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.08145963465472954]
    }
  }
}


#### gRPC Request

In [55]:
grpc_request_ambassador("mymab",API_AMBASSADOR)

meta {
  puid: "crok3pdntkt5u97shm19g1s8fj"
  routing {
    key: "eg-router"
    value: 1
  }
  requestPath {
    key: "classifier-2"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.06808804098089556
  }
}



In [56]:
!helm delete mymab --purge

release "mymab" deleted


## Serve Single Model with Outlier Detector

In [63]:
!helm install ../helm-charts/seldon-single-model --name mymodel --set outlier_detection.enabled=true 

NAME:   mymodel
LAST DEPLOYED: Sun Oct  7 10:35:02 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME     AGE
mymodel  0s




In [64]:
!helm template ../helm-charts/seldon-single-model --name mymodel --set outlier_detection.enabled=true | pygmentize -l json

---
# Source: seldon-single-model/templates/model.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mymodel"
    },
    "spec": {
        "name": "mymodel",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                        "terminationGracePeriodSecond

In [66]:
!kubectl rollout status deploy/mymodel-mymodel-classifier-0
!kubectl rollout status deploy/mymodel-mymodel-svc-orch
!kubectl rollout status deploy/mymodel-mymodel-outlier-detector-1

Waiting for deployment "mymodel-mymodel-classifier-0" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-mymodel-classifier-0" successfully rolled out
Waiting for deployment "mymodel-mymodel-svc-orch" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-mymodel-svc-orch" successfully rolled out
Waiting for deployment "mymodel-mymodel-outlier-detector-1" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymodel-mymodel-outlier-detector-1" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymodel-mymodel-outlier-detector-1" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment 

### Get predictions

In [67]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [68]:
rest_request_ambassador("mymodel",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "idb4ivgjblcv8tb8s0v53f52fa",
    "tags": {
      "outlierScore": [0.0]
    },
    "routing": {
      "outlier-detector": -1
    },
    "requestPath": {
      "outlier-detector": "seldonio/outlier_mahalanobis:0.3",
      "classifier": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.08804202057605118]
    }
  }
}


#### gRPC Request

In [69]:
grpc_request_ambassador("mymodel",API_AMBASSADOR)

meta {
  puid: "abh4ehrj1ssm59i8gpkgr2ffj2"
  tags {
    key: "outlierScore"
    value {
      list_value {
        values {
          number_value: 0.0
        }
      }
    }
  }
  routing {
    key: "outlier-detector"
    value: -1
  }
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "outlier-detector"
    value: "seldonio/outlier_mahalanobis:0.3"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.077437681875211
  }
}



In [70]:
!helm delete mymodel --purge

release "mymodel" deleted
